In [93]:
import pandas as pd
import polars as pl
import plotnine as pn
import numpy as np
from tqdm.notebook import tqdm

import neuralforecast
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NBEATSx, NHITS
from neuralforecast.auto import AutoNBEATS, AutoNHITS
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.pipeline import FeatureUnion, make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import (
    LabelEncoder,
    StandardScaler,
    OneHotEncoder,
    FunctionTransformer,
)
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import config_context
import optuna
import os
import holidays

%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


### Data Preparation

Focus on CH spot price first

In [94]:
df = pd.read_csv(
    "../00 Data Retrieval and Cleaning/0_df_final_ml_predictive.csv",
    parse_dates=["date"],
)
df.head()

,date,auction_price_ch_de,auction_price_de_ch,dst,day_ahead_price_at,day_ahead_price_ch,day_ahead_price_de,day_ahead_price_fr,day_ahead_price_it,actual_load_at,...,wind_offshore_actual_aggregated_it,wind_offshore_forecast_de,wind_onshore_actual_aggregated_at,wind_onshore_actual_aggregated_de,wind_onshore_actual_aggregated_fr,wind_onshore_actual_aggregated_it,wind_onshore_ch,wind_onshore_forecast_at,wind_onshore_forecast_de,wind_onshore_forecast_fr
0,2023-01-01 00:00:00+00:00,0.01,6.09,1,NaN,-7.25,-1.07,NaN,NaN,NaN,...,NaN,3390.25,NaN,NaN,NaN,NaN,NaN,1174.0,35415.50,13933.0
1,2023-01-01 01:00:00+00:00,0.01,5.50,1,NaN,-3.99,-1.47,NaN,NaN,NaN,...,NaN,3395.50,NaN,NaN,NaN,NaN,NaN,1194.0,35146.75,13583.0
2,2023-01-01 02:00:00+00:00,0.01,6.45,1,NaN,-7.71,-5.08,NaN,NaN,NaN,...,NaN,3410.25,NaN,NaN,NaN,NaN,NaN,1085.0,34449.00,13230.0
3,2023-01-01 03:00:00+00:00,0.01,9.08,1,NaN,-9.71,-4.49,NaN,NaN,NaN,...,NaN,3431.25,NaN,NaN,NaN,NaN,NaN,897.0,33905.25,12877.0
4,2023-01-01 04:00:00+00:00,0.01,13.33,1,NaN,-15.15,-5.40,NaN,NaN,NaN,...,NaN,3454.25,NaN,NaN,NaN,NaN,NaN,697.0,33362.75,12311.0


Need to lag the other three target variables by 24 hours, haven't done that yet:

In [95]:
df = (
    df
    .assign(
        day_ahead_price_de=lambda x: x["day_ahead_price_de"].shift(24),
        auction_price_de_ch=lambda x: x["auction_price_de_ch"].shift(24),
        auction_price_ch_de=lambda x: x["auction_price_ch_de"].shift(24),
    )
)

### Additional Feature Generation

- There might be a benefit of encoding cyclical calendar information
- Additionally: Holidays

Include a trend column:

In [96]:
df = df.assign(trend=lambda x: x.index, unique_id="spot_ch")

In [97]:
# Define the country (Switzerland)
country = "CH"

regional_holidays = holidays.CH(
    years=df.date.dt.year.unique().tolist()
)

In [98]:
holiday_df = pd.DataFrame(
    {
        "holiday_name": list(regional_holidays.values()),
        "holiday_date": list(regional_holidays.keys()),
    }
)

holiday_df.sort_values("holiday_date").head()

,holiday_name,holiday_date
4,Neujahrestag,2023-01-01
5,Auffahrt,2023-05-18
6,Nationalfeiertag,2023-08-01
7,Weihnachten,2023-12-25
0,Neujahrestag,2024-01-01


In [99]:
holiday_df.value_counts("holiday_name")

holiday_name
Auffahrt            2
Nationalfeiertag    2
Neujahrestag        2
Weihnachten         2
Name: count, dtype: int64

In [100]:
df = (
    df.assign(
        hour=lambda x: x.date.dt.hour + 1,
        month=lambda x: x.date.dt.month,
        quarter=lambda x: x.date.dt.quarter,
        wday=lambda x: x.date.dt.day_of_week + 1,
        weekend=lambda x: np.where(
            x.date.dt.day_name().isin(["Sunday", "Saturday"]), 1, 0
        ),
        work_hour=lambda x: np.where(
            x["hour"].isin(np.arange(17, 24).tolist() + np.arange(1, 5).tolist()), 0, 1
        ),
        week_hour=lambda x: x.date.dt.dayofweek * 24 + (x.date.dt.hour + 1),
        year=lambda x: x.date.dt.year,
        hour_counter=lambda x: np.arange(0, x.shape[0]),
    )
    .assign(day=lambda x: x.date.dt.date)
    .merge(holiday_df, how="left", left_on="day", right_on="holiday_date")
    .drop(["holiday_date", "day"], axis=1)
    .assign(
        holiday_name=lambda x: np.where(
            x["holiday_name"].isna(), "none", x["holiday_name"]
        )
    )
)

In [101]:
df.value_counts("holiday_name")

holiday_name
none                9381
Neujahrestag          48
Auffahrt              24
Nationalfeiertag      24
Weihnachten           24
Name: count, dtype: int64

### Feature Engineering

Other:
- `date`: drop, can't feed into net

Numerical:
- everything but `holiday_name`

Categorical
- `holiday_name`: one-hot encode

#### Cyclical Encoding

- avoid issue with exploding feature space when one-hot encoding hundreds of levels in categorical vars
- puts end of cycle closer to beginning (End of Year is not that different from BOY)

In [102]:
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

In [103]:
# hour in day
df["hour_sin"] = sin_transformer(24).fit_transform(df["hour"].astype(float))
df["hour_cos"] = cos_transformer(24).fit_transform(df["hour"].astype(float))

# hour in week
df["week_hour_sin"] = sin_transformer(168).fit_transform(df["week_hour"].astype(float))
df["week_hour_cos"] = cos_transformer(168).fit_transform(df["week_hour"].astype(float))

# month
df["month_sin"] = sin_transformer(12).fit_transform(df["month"].astype(float))
df["month_cos"] = cos_transformer(12).fit_transform(df["month"].astype(float))

# quarter
df["quarter_sin"] = sin_transformer(4).fit_transform(df["quarter"].astype(float))
df["quarter_cos"] = cos_transformer(4).fit_transform(df["quarter"].astype(float))

# weekday
df["wday_sin"] = sin_transformer(7).fit_transform(df["wday"].astype(float))
df["wday_cos"] = cos_transformer(7).fit_transform(df["wday"].astype(float))

df = df.drop(["hour", "month", "quarter", "wday", "week_hour"], axis=1)

#### `sklearn` Pipeline for Data Preparation

In [104]:
manual_cols = ["trend", "unique_id"]
drop_cols = ["date", "day_ahead_price_ch"]

pipeline_cols = [
    col
    for col in df.drop(drop_cols, axis=1).columns
    if col not in manual_cols
]

num_cols = (
    df.drop(drop_cols, axis=1)
    .filter(pipeline_cols)
    .select_dtypes(include=np.number)
    .columns
)
cat_cols = (
    df.drop(drop_cols, axis=1)
    .filter(pipeline_cols)
    .select_dtypes(exclude=np.number)
    .columns
)

In [105]:
pipeline_cols

['auction_price_ch_de',
 'auction_price_de_ch',
 'dst',
 'day_ahead_price_at',
 'day_ahead_price_de',
 'day_ahead_price_fr',
 'day_ahead_price_it',
 'actual_load_at',
 'actual_load_ch',
 'actual_load_de',
 'actual_load_fr',
 'actual_load_it',
 'allocated_capacity_ch_de',
 'allocated_capacity_de_ch',
 'biomass_actual_aggregated_at',
 'biomass_actual_aggregated_de',
 'biomass_actual_aggregated_fr',
 'biomass_actual_aggregated_it',
 'capacity_forecast_at_ch',
 'capacity_forecast_ch_at',
 'capacity_forecast_ch_de_lu',
 'capacity_forecast_ch_fr',
 'capacity_forecast_ch_it',
 'capacity_forecast_de_lu_ch',
 'capacity_forecast_fr_ch',
 'capacity_forecast_it_ch',
 'crossborder_actual_flow_at_ch',
 'crossborder_actual_flow_ch_at',
 'crossborder_actual_flow_ch_de_lu',
 'crossborder_actual_flow_ch_fr',
 'crossborder_actual_flow_ch_it',
 'crossborder_actual_flow_de_lu_ch',
 'crossborder_actual_flow_fr_ch',
 'crossborder_actual_flow_it_ch',
 'fossil_brown_coal_lignite_actual_aggregated_de',
 'fossil

In [106]:
num_cols

Index(['auction_price_ch_de', 'auction_price_de_ch', 'dst',
       'day_ahead_price_at', 'day_ahead_price_de', 'day_ahead_price_fr',
       'day_ahead_price_it', 'actual_load_at', 'actual_load_ch',
       'actual_load_de',
       ...
       'hour_sin', 'hour_cos', 'week_hour_sin', 'week_hour_cos', 'month_sin',
       'month_cos', 'quarter_sin', 'quarter_cos', 'wday_sin', 'wday_cos'],
      dtype='object', length=116)

In [107]:
cat_cols

Index(['holiday_name'], dtype='object')

In [108]:
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer()),
        # ("scaler", StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        (
            "encoder",
            OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        ),
    ]
)

# Making column transformer where all transformers in the pipelines are included
preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, num_cols),
        ("categorical", categorical_transformer, cat_cols),
    ],
    remainder="passthrough",
)

### Preprocess the data

In [109]:
train_end = pd.Timestamp("2023-09-01").tz_localize("UTC")
val_end = pd.Timestamp("2024-01-01").tz_localize("UTC")

# Create splits
df_train = df.query("date < @val_end")
# df_val = df.query("date < @val_end").query("ds >= @train_end").head(500)
df_test = df.query("date >= @val_end")

In [110]:
X_train = df_train.drop(columns=["date", "day_ahead_price_ch"])
y_train = df_train["day_ahead_price_ch"]

X_test = df_test.drop(columns=["date", "day_ahead_price_ch"])
y_test = df_test["day_ahead_price_ch"]

In [111]:
fitted_preprocessor = preprocessor.fit(X_train)

In [112]:
X_train_preprocessed = pd.DataFrame(
    fitted_preprocessor.transform(X_train),
    columns=fitted_preprocessor.get_feature_names_out(),
)
X_test_preprocessed = pd.DataFrame(
    fitted_preprocessor.transform(X_test),
    columns=fitted_preprocessor.get_feature_names_out(),
)

In [113]:
# Replace prefixes in column names
new_cols = X_train_preprocessed.columns.str.replace('numeric__', '').str.replace('categorical__', '').str.replace('remainder__', '')

# Assign new column names to the DataFrame
X_train_preprocessed.columns = new_cols
X_test_preprocessed.columns = new_cols

In [114]:
X_train_preprocessed.head()

,auction_price_ch_de,auction_price_de_ch,dst,day_ahead_price_at,day_ahead_price_de,day_ahead_price_fr,day_ahead_price_it,actual_load_at,actual_load_ch,actual_load_de,...,quarter_cos,wday_sin,wday_cos,holiday_name_Auffahrt,holiday_name_Nationalfeiertag,holiday_name_Neujahrestag,holiday_name_Weihnachten,holiday_name_none,trend,unique_id
0,1.337356,14.528225,1.0,102.39752,95.427349,97.109492,127.814714,6637.036266,6970.689146,52843.424376,...,0.0,-0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,spot_ch
1,1.337356,14.528225,1.0,102.39752,95.427349,97.109492,127.814714,6637.036266,6970.689146,52843.424376,...,0.0,-0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,spot_ch
2,1.337356,14.528225,1.0,102.39752,95.427349,97.109492,127.814714,6637.036266,6970.689146,52843.424376,...,0.0,-0.0,1.0,0.0,0.0,1.0,0.0,0.0,2,spot_ch
3,1.337356,14.528225,1.0,102.39752,95.427349,97.109492,127.814714,6637.036266,6970.689146,52843.424376,...,0.0,-0.0,1.0,0.0,0.0,1.0,0.0,0.0,3,spot_ch
4,1.337356,14.528225,1.0,102.39752,95.427349,97.109492,127.814714,6637.036266,6970.689146,52843.424376,...,0.0,-0.0,1.0,0.0,0.0,1.0,0.0,0.0,4,spot_ch


In [115]:
df_train = pd.concat(
    [
        df_train["date"].reset_index(drop=True),
        y_train.reset_index(drop=True),
        X_train_preprocessed.reset_index(drop=True),
    ],
    axis=1,
)
df_test = pd.concat(
    [
        df_test["date"].reset_index(drop=True),
        y_test.reset_index(drop=True),
        X_test_preprocessed.reset_index(drop=True),
    ],
    axis=1,
)

### Training an initial model for testing purposes

In [116]:
df_train = df_train.rename(columns={"day_ahead_price_ch": "y", "date": "ds"})
df_test = df_test.rename(columns={"day_ahead_price_ch": "y", "date": "ds"})

df_train.head()

,ds,y,auction_price_ch_de,auction_price_de_ch,dst,day_ahead_price_at,day_ahead_price_de,day_ahead_price_fr,day_ahead_price_it,actual_load_at,...,quarter_cos,wday_sin,wday_cos,holiday_name_Auffahrt,holiday_name_Nationalfeiertag,holiday_name_Neujahrestag,holiday_name_Weihnachten,holiday_name_none,trend,unique_id
0,2023-01-01 00:00:00+00:00,-7.25,1.337356,14.528225,1.0,102.39752,95.427349,97.109492,127.814714,6637.036266,...,0.0,-0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,spot_ch
1,2023-01-01 01:00:00+00:00,-3.99,1.337356,14.528225,1.0,102.39752,95.427349,97.109492,127.814714,6637.036266,...,0.0,-0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,spot_ch
2,2023-01-01 02:00:00+00:00,-7.71,1.337356,14.528225,1.0,102.39752,95.427349,97.109492,127.814714,6637.036266,...,0.0,-0.0,1.0,0.0,0.0,1.0,0.0,0.0,2,spot_ch
3,2023-01-01 03:00:00+00:00,-9.71,1.337356,14.528225,1.0,102.39752,95.427349,97.109492,127.814714,6637.036266,...,0.0,-0.0,1.0,0.0,0.0,1.0,0.0,0.0,3,spot_ch
4,2023-01-01 04:00:00+00:00,-15.15,1.337356,14.528225,1.0,102.39752,95.427349,97.109492,127.814714,6637.036266,...,0.0,-0.0,1.0,0.0,0.0,1.0,0.0,0.0,4,spot_ch


In [117]:
# num_hidden = 3
# mlp_units_n = 64
# mlp_units=[[mlp_units_n, mlp_units_n]]*num_hidden

In [120]:
model = NHITS(
    h=38,
    input_size=336,
    loss=MSE(),
    scaler_type="standard",
    max_steps=500,
    val_check_steps=10,
    early_stop_patience_steps=3,
    learning_rate=0.0001,
    batch_size=32,
    futr_exog_list=df_train.drop(columns=["y", "unique_id", "ds"]).columns.tolist(),
)

Seed set to 1


In [121]:
nf = NeuralForecast(models=[model], freq="h")
nf.fit(df=df_train, val_size=int(df_train.shape[0]*0.2))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MSE           | 0     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 50.0 M
-----------------------------------------------
50.0 M    Trainable params
0         Non-trainable params
50.0 M    Total params
199.934   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

In [ ]:
nf.predict(futr_df=df_test, step_size=1).reset_index()

c:\Users\mathi\miniconda3\envs\statslab\lib\site-packages\neuralforecast\core.py:758: UserWarning: Dropped 705 unused rows from `futr_df`.
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

c:\Users\mathi\miniconda3\envs\statslab\lib\site-packages\neuralforecast\core.py:196: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.


,unique_id,ds,NHITS
0,spot_ch,2024-01-01 00:00:00+00:00,2646.242188
1,spot_ch,2024-01-01 01:00:00+00:00,2777.308594
2,spot_ch,2024-01-01 02:00:00+00:00,1899.793945
3,spot_ch,2024-01-01 03:00:00+00:00,1618.281250
4,spot_ch,2024-01-01 04:00:00+00:00,2187.997070
5,spot_ch,2024-01-01 05:00:00+00:00,581.361389
6,spot_ch,2024-01-01 06:00:00+00:00,-445.664062
7,spot_ch,2024-01-01 07:00:00+00:00,-2099.810547
8,spot_ch,2024-01-01 08:00:00+00:00,-4157.234375
9,spot_ch,2024-01-01 09:00:00+00:00,-4850.921875
